<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/KISTA_AUTOMATIC1111_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title # Kista AUTOMATIC1111 WebUI - v1.0

#@markdown This Colab is maintained by [github.com/steinhaug/stable-diffusion](https://github.com/steinhaug/stable-diffusion/) and lets you load Automatic1111 and Stable Diffusion with a Colab notebook. Play this cell to verify if you are running with a GPU and how much RAM.

#@markdown <a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/SD_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Fri Mar  3 17:48:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title Load and verify settings
#@markdown ![Load your config](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/colab-split-load.png)

from pathlib import Path
from IPython.display import display, Markdown, clear_output
from ipywidgets import widgets, GridspecLayout, Layout
from configparser import ConfigParser

inip = Path("/content/drive/MyDrive")
if not inip.is_dir():
    from google.colab import drive
    drive.mount('/content/drive')
    clear_output()
else:
    print("Drive already mounted, continue.");
    clear_output()

INI_FILE_LOC = "/content/drive/MyDrive/sd-colab.ini"

def strtobool (val):
    """Quick cast tro boolean
    """
    val = val.lower()
    if val in ('y', 'yes', 't', 'true', 'on', '1'):
        return True
    elif val in ('', 'n', 'no', 'f', 'false', 'off', '0'):
        return False
    else:
        raise ValueError("invalid truth value %r" % (val,))

def getValue (savedconf, key, type):
    try:
        val = savedconf[key]
    except KeyError:
        val = ""
    if type == 'bool':
        return strtobool(val)
    else:
        return val

ext_controlnet = widgets.Checkbox( value=False, description='ControlNet and Open Pose', disabled=False, indent=False )
ext_civitai = widgets.Checkbox( value=False, description='Image and civitai', disabled=False, indent=False )
ext_huggingface = widgets.Checkbox( value=False, description='Huggingface weui', disabled=False, indent=False )

ext_inspiration = widgets.Checkbox( value=False, description='Inspiration', disabled=False, indent=False )
ext_artists_to_study = widgets.Checkbox( value=False, description='Artists to study', disabled=False, indent=False )

modat_controlnet = widgets.Checkbox( value=False, description='ControlNet', disabled=False, indent=False )
modat_inspiration = widgets.Checkbox( value=False, description='Inspiration imgs', disabled=False, indent=False )

path_cpkt = widgets.Text( value="", placeholder='Path to main checkpoint file', disabled=False )
dir_cpkt = widgets.Text( value="", placeholder='Path to checkpoints folder', disabled=False )
dir_controlnet = widgets.Text( value="", placeholder='Path to controlnet models folder', disabled=False )
dir_outdir_samples = widgets.Text( value="", placeholder='Save all generated images path', disabled=False )
dir_outdir_grids = widgets.Text( value="", placeholder='Save all generated images path', disabled=False )

ckpt_15 = widgets.Checkbox( value=False, description='v1.5', disabled=False, indent=False )
ckpt_15inpaint = widgets.Checkbox( value=False, description='v1.5 inpaint', disabled=False, indent=False )
ckpt_instruct_pix2pix = widgets.Checkbox( value=False, description='InstructPix2Pix', disabled=False, indent=False )
ckpt_21 = widgets.Checkbox( value=False, description='v2.1', disabled=False, indent=False )

def btn_continue_event_handler(btn_object):
    grid.close();
    print('You pressed {}'.format(btn_object.description))
    print('Creating .ini')
    config_object = ConfigParser()
    config_object["CONF"] = {
        "ext_controlnet": ext_controlnet.value,
        "ext_civitai": ext_civitai.value,
        "ext_huggingface": ext_huggingface.value,
        "ext_inspiration": ext_inspiration.value,
        "ext_artists_to_study": ext_artists_to_study.value,
        "path_cpkt": path_cpkt.value,
        "dir_cpkt": dir_cpkt.value,
        "dir_controlnet": dir_controlnet.value,
        "dir_outdir_samples": dir_controlnet.value,
        "dir_outdir_grids": dir_outdir_grids.value,
        "ckpt_15": ckpt_15.value,
        "ckpt_15inpaint": ckpt_15inpaint.value,
        "ckpt_instruct_pix2pix": ckpt_instruct_pix2pix.value,
        "ckpt_21": ckpt_21.value,
        "modat_controlnet": modat_controlnet.value,
        "modat_inspiration": modat_inspiration.value,
    }
    with open(INI_FILE_LOC, 'w') as conf:
        config_object.write(conf)
    clear_output(); print('[1;32m Settings saved and confirmed! ✓')

# - - - - - - 
btn_continue = widgets.Button(description='Save configuration and continue', button_style='info', layout=Layout(width='auto', height='auto'))
btn_continue.on_click(btn_continue_event_handler)

btn1 = widgets.Button(description='EXTENSIONS', layout=Layout(width='auto', height='auto'))
btn1.style.button_color='White'
btn1.style.font_weight='bold'
btn2 = widgets.Button(description='PATHS', layout=Layout(width='auto', height='auto'))
btn2.style.button_color='White'
btn2.style.font_weight='bold'
btn3 = widgets.Button(description='CHECKPOINTS', layout=Layout(width='auto', height='auto'))
btn3.style.button_color='White'
btn3.style.font_weight='bold'
btn4 = widgets.Button(description='MODELS', layout=Layout(width='auto', height='auto'))
btn4.style.button_color='White'
btn4.style.font_weight='bold'

grid = GridspecLayout(7, 5)

grid[0, 0] = btn1
grid[1, 0] = ext_controlnet
grid[2, 0] = ext_civitai
grid[3, 0] = ext_huggingface

grid[1, 1] = ext_inspiration
grid[2, 1] = ext_artists_to_study

grid[0, 2] = btn3
grid[1, 2] = ckpt_15
grid[2, 2] = ckpt_15inpaint
grid[3, 2] = ckpt_instruct_pix2pix
grid[4, 2] = ckpt_21

grid[0, 3] = btn4
grid[1, 3] = modat_controlnet
grid[2, 3] = modat_inspiration

grid[0, 4] = btn2
grid[1, 4] = path_cpkt
grid[2, 4] = dir_cpkt
grid[3, 4] = dir_controlnet
grid[4, 4] = dir_outdir_samples
grid[5, 4] = dir_outdir_grids
grid[6, :] = btn_continue

#display(Markdown('**EXTENSIONS**'))
display(grid)

inip = Path(INI_FILE_LOC)
if inip.is_file():
    # Load values
    config_object = ConfigParser()
    config_object.read(INI_FILE_LOC)
    savedconf = config_object["CONF"]
  
    ext_controlnet.value = getValue(savedconf, 'ext_controlnet', 'bool')
    ext_civitai.value = getValue(savedconf, 'ext_civitai', 'bool')
    ext_huggingface.value = getValue(savedconf, 'ext_huggingface', 'bool')
    ext_inspiration.value = getValue(savedconf, 'ext_inspiration', 'bool')
    ext_artists_to_study.value = getValue(savedconf, 'ext_artists_to_study', 'bool')
    modat_controlnet.value = getValue(savedconf, 'modat_controlnet', 'bool')
    modat_inspiration.value = getValue(savedconf, 'modat_inspiration', 'bool')
    path_cpkt.value = getValue(savedconf, 'path_cpkt', 'string')
    dir_cpkt.value = getValue(savedconf, 'dir_cpkt', 'string')
    dir_controlnet.value = getValue(savedconf, 'dir_controlnet', 'string')
    dir_outdir_samples.value = getValue(savedconf, 'dir_outdir_samples', 'string')
    dir_outdir_grids.value = getValue(savedconf, 'dir_outdir_grids', 'string')
    ckpt_15.value = getValue(savedconf, 'ckpt_15', 'bool')
    ckpt_15inpaint.value = getValue(savedconf, 'ckpt_15inpaint', 'bool')
    ckpt_instruct_pix2pix.value = getValue(savedconf, 'ckpt_instruct_pix2pix', 'bool')
    ckpt_21.value = getValue(savedconf, 'ckpt_21', 'bool')
    # https://tutswiki.com/read-write-config-files-in-python/






 Settings saved and confirmed! ✓


In [ ]:
#@title Install and launch the Automatic1111 WebUi
#@markdown ![Load your config](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/colab-split-start.png)



skip_install = True #@param {type:"boolean"}
#@markdown When you have installed the system, or for some reason restart the process you can check the above to just launch the installed software.

from IPython.display import clear_output;
%cd /content/
clear_output


if not skip_install:
    print('[1;32mStarting install...')  
    %pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if ext_controlnet.value:
        !git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
        !git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
    if modat_controlnet.value:
        !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_openpose.pth
        !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_depth.pth
        !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_scribble.pth

    if ext_civitai.value:
        !git clone https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
        !git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser

    if ext_huggingface.value:
        !git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
        !pip install huggingface-hub==0.11.1

    if ext_inspiration.value:
        !git clone https://github.com/yfszzx/stable-diffusion-webui-inspiration /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration
    if modat_inspiration.value:
        !wget https://huggingface.co/datasets/yfszzx/inspiration/resolve/main/inspiration.zip -O /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip
        from zipfile import ZipFile
        with ZipFile("/content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip", 'r') as zObject:
            zObject.extractall(path="/content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration")
        !rm /content/stable-diffusion-webui/extensions/stable-diffusion-webui-inspiration/inspiration.zip

    if ext_artists_to_study.value:
        !git clone https://github.com/camenduru/stable-diffusion-webui-artists-to-study /content/stable-diffusion-webui/extensions/stable-diffusion-webui-artists-to-study

    if ckpt_15.value:
        !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/blob/main/v1-5-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5.ckpt
        !wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5.vae.pt
        #!wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/vae-ft-mse-840000-ema-pruned.ckpt

    if ckpt_15inpaint.value:
        !wget https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5-inpainting.ckpt

    if ckpt_instruct_pix2pix.value:
        !wget https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/instruct-pix2pix.ckpt

    if ckpt_21.value:
        !wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768.ckpt
    print('[1;32mInstalling complete!')
else:
    print('[1;32mSkipping installers... done!')  


%cd /content/stable-diffusion-webui/


if not skip_install:
    inip = Path('/content/stable-diffusion-webui/conf.json')
    if not inip.is_file():
        print('[1;32mCreating WebUI conf.json')
        conf_ini_json = {
            "outdir_samples": dir_outdir_samples.value,
            "outdir_grids": dir_outdir_samples.value,
            "control_net_models_path": modat_controlnet.value,
            "hide_samplers": [
                "LMS",
                "Heun",
                "DPM2",
                "DPM2 a",
                "DPM++ 2S a",
                "DPM++ 2M",
                "DPM++ SDE",
                "DPM fast",
                "DPM adaptive",
                "LMS Karras",
                "PLMS",
                "DPM++ 2S a Karras",
                "DPM2 a Karras",
                "DPM2 Karras"
            ],
            "unload_models_when_training": True,
            "training_xattention_optimizations": True
        }
        with open("conf.json", "w") as jsonfile:
            json.dump(conf_ini_json, jsonfile)

args = "--share --xformers --enable-insecure-extension-access --ui-settings-file=\"conf.json\""

if skip_install:
    args = args + " --skip-install"
if path_cpkt.value != "":
    args = args + " --ckpt=\"" + path_cpkt.value + "\""
if dir_cpkt.value != "":
    args = args + " --ckpt-dir=\"" + dir_cpkt.value + "\""

print('[1;32mLaunching WebUI')  
!python launch.py $args



/content
Skipping installers... done!
/content/stable-diffusion-webui
Launching WebUI
Python 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
Commit hash: 0cc0ee1bcb4c24a8c9715f66cede06601bfc00c8
Cloning Stable Diffusion into repositories/stable-diffusion-stability-ai...
Cloning Taming Transformers into repositories/taming-transformers...
